# HuggingTweets - Tweet Generation with Huggingface

*Disclaimer: this demo is not to be used to publish any false generated information but to perform research on Natural Language Generation (NLG).*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/borisdayma/huggingtweets/blob/master/huggingtweets.ipynb)

## Introduction

Generating realistic text has become more and more efficient with models such as [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf). Those models are trained on very large datasets and require heavy computer resources (and time!).

However, we can use Transfer Learning and a single GPU to quickly fine-tune a pre-trained model on a given task.

We test if we can imitate the writing style of a Twitter user by only using some of his tweets. Twitter API let us download "only" the 3200 most recent tweets from any single user, which we then filter out (to remove retweets, short content, etc).

[HuggingFace](https://huggingface.co/) gives us an easy access to pre-trained models and fine-tuning techniques for Natural Language Generation (NLG) tasks.

We will be monitoring the training with [W&B](https://docs.wandb.com/huggingface) (which is integrated in HuggingFace) to ensure the model is learning from the data and compare multiple experiments.

![](https://i.imgur.com/vnejHGh.png)

## Install dependencies

In [ ]:
# Uncomment if those libraries are not installed
!pip install torch -qq
!pip install git+https://github.com/huggingface/transformers.git -qq
!pip install wandb -qq
!pip install tweepy -qq

In [ ]:
# Huggingface scripts for fine-tuning models and language generation
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py -q
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py -q

## Set up a Twitter Development Account

In order to access Twitter data, we need to:

* [create a Twitter development account](https://developer.twitter.com/en/apply-for-access)
* [create a Twitter app](https://developer.twitter.com/en/apps)
* get your consumer API keys: "API key" and "API secret key"

The entire process only takes a few minutes.

In [ ]:
# <--- Enter your credentials (don't share with anyone) --->
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXX'

## Download tweets from a user

We download latest tweets associated to a user account through [Tweepy](http://docs.tweepy.org/).

In [ ]:
import tweepy

In [ ]:
# authenticate
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

We grab all available tweets (limited to 3200 per API limitations) based on Twitter handle.

**Note**: Protected users may only be requested when the authenticated user either "owns" the timeline or is an approved follower of the owner.

In [ ]:
# <--- Enter the screen name of the user you will download your dataset from --->
handle = 'l2k'

In [ ]:
# Adapted from https://gist.github.com/onmyeoin/62c72a7d61fc840b2689b2cf106f583c

# initialize a list to hold all the tweepy Tweets & list with no retweets
alltweets = []

# make initial request for most recent tweets with extended mode enabled to get full tweets
new_tweets = api.user_timeline(
    screen_name=handle, tweet_mode='extended', count=200)

# save most recent tweets
alltweets.extend(new_tweets)

# save the id of the oldest tweet less one
oldest = alltweets[-1].id - 1

# check we cannot get more tweets
no_tweet_count = 0

# keep grabbing tweets until the api limit is reached
while True:
    print(f'getting tweets before id {oldest}')

    # all subsequent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(
        screen_name=handle, tweet_mode='extended', count=200, max_id=oldest)
    
    # stop if no more tweets (try a few times as they sometimes eventually come)
    if not new_tweets:
        no_tweet_count +=1
    else:
        no_tweet_count = 0
    if no_tweet_count > 5: break

    # save most recent tweets
    alltweets.extend(new_tweets)

    # update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    print(f'...{len(alltweets)} tweets downloaded so far')

## Create a dataset from downloaded tweets

We remove:
* retweets (since it's not in the wording style of target author)
* tweets with no interesting content (limited to url's, user mentionss, "thank you"…)

We clean up remaining tweets:
* we remove url's
* we replace "@" mentions with user names

In [ ]:
import random
import re
import torch

In [ ]:
class user_handle:
    'Get a user handle and cache it to avoid calling too much twitter api.'
    handles = {}
    def get(handle):
        if handle not in user_handle.handles.keys():            
            try:
                user_handle.handles[handle] = api.get_user(handle).name
            except:
                user_handle.handles[handle] = None
        return user_handle.handles[handle]

In [ ]:
def replace_handle(word):
    'Replace user handles, remove "@" and "#"'
    if word[0] == '@':
        handle = re.search('^@(\w)+', word)
        if handle:
            user = user_handle.get(handle.group())
            if user is not None: return user + word[handle.endpos:]
    return word

In [ ]:
def keep_tweet(tweet):
    'Return true if not a retweet'
    if hasattr(tweet, 'retweeted_status'):
        return False
    return True

In [ ]:
def curate_tweets(tweets):
    'Decide which tweets we keep and replace handles'
    curated_tweets = []
    for tweet in tweets:
        if keep_tweet(tweet):
            curated_tweets.append(' '.join(replace_handle(w) for w in tweet.full_text.split()))
    return curated_tweets

In [ ]:
curated_tweets = curate_tweets(alltweets)

We verify our list of tweets is well curated.

In [ ]:
print(f'Total number of tweets: {len(alltweets)}\nCurated tweets: {len(curated_tweets)}')

In [ ]:
print('Original tweets\n')
for t in alltweets[:5]:
    print(f'{t.full_text}\n')

In [ ]:
print('Curated tweets\n')
for t in curated_tweets[:5]:
    print(f'{t}\n')

We remove boring tweets (tweets with only urls or too short) and cleanup texts.

In [ ]:
def cleanup_tweet(tweet):
    "Clean tweet text"
    text = ' '.join(t for t in tweet.split() if 'http' not in t)
    text = text.replace('&amp;', 'and')
    if text.split() and text.split()[0] == '.':
         text = ' '.join(text.split()[1:])
    return text

In [ ]:
def boring_tweet(tweet):
    "Check if this is a boring tweet"
    boring_stuff = ['http', '@', '#', 'thank', 'thanks', 'I', 'you']
    if len(tweet.split()) < 3:
        return True
    if all(any(bs in t.lower() for bs in boring_stuff) for t in tweet):
        return True
    return False

In [ ]:
clean_tweets = [cleanup_tweet(t) for t in curated_tweets]
cool_tweets = [tweet for tweet in clean_tweets if not boring_tweet(tweet)]
print(f'Curated tweets: {len(curated_tweets)}\nCool tweets: {len(cool_tweets)}')

In [ ]:
# OTHER VERSION
handle = "vanpelt"
import json
import urllib3
http = urllib3.PoolManager(retries=urllib3.Retry(3))
res = http.request("GET", f"https://us-central1-playground-111.cloudfunctions.net/tweets_http?handle={handle}")
curated_tweets = json.loads(res.data.decode('utf-8')) 

We split data into training and validation sets (90/10).

In [ ]:
# shuffle data
random.shuffle(cool_tweets)

# fraction of training data
split_train_valid = 0.9

# split dataset
train_size = int(split_train_valid * len(cool_tweets))
valid_size = len(cool_tweets) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(cool_tweets, [train_size, valid_size])

In [ ]:
# OTHER VERSION
with open('{}_train.txt'.format(handle), 'w') as f:
    f.write('\n'.join(cool_tweets))

We export our datasets as text files.

In [ ]:
with open('{}_train.txt'.format(handle), 'w') as f:
    f.write('\n'.join(train_dataset))

with open('{}_valid.txt'.format(handle), 'w') as f:
    f.write('\n'.join(valid_dataset))

## Log and monitor training through W&B

In order to check our model is training correctly and compare experiments, we are going to use the W&B integration from huggingface.

### [Sign up for a free account →](https://app.wandb.ai/login?signup=true)

- **Unified dashboard**: Central repository for all your model metrics and predictions
- **Lightweight**: No code changes required to integrate with Hugging Face
- **Accessible**: Free for individuals and academic teams
- **Secure**: All projects are private by default
- **Trusted**: Used by machine learning teams at OpenAI, Toyota, GitHub, Lyft and more

### API Key
Once you've signed up, run the next cell and click on the link to get your API key and authenticate this notebook.

In [ ]:
import wandb
wandb.login()

## Fine-tuning the model

HuggingFace includes the script `run_language_modeling` making it easy to fine-tune a pre-trained model.

We use a pre-trained GPT-2 model and fine-tune it on our dataset.

Training is automatically logged on W&B (see [documentation](https://docs.wandb.com/huggingface)). Urls are generated to visualize ongoing runs or you can just open your [dashboard](http://app.wandb.ai/).

I quickly tested running for several epochs and my run was showing I started overfitting after 4 epochs so this is the limit I use to fine-tune my model (takes less than 2 minutes).

![](https://i.imgur.com/1uIxLFe.png)

In [ ]:
# Associate run to a project (optional)
%env WANDB_PROJECT=huggingface-twitter-2

We use HuggingFace script `run_language_modeling.py` to fine-tune our model (see [doc](https://huggingface.co/transformers/)).

In [ ]:
!python run_language_modeling.py \
    --output_dir=output/$handle \
    --overwrite_output_dir \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=$handle\_train.txt \
    --do_eval --eval_data_file=$handle\_valid.txt \
    --evaluate_during_training \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 4

## Let's test our trained model!

We test our model on a few sample sentences.

In [ ]:
SENTENCES = ["I think that",
            "I like",
            "I don't like",
            "I want"]

We use HuggingFace script `run_generation.py` to generate sentences (see [doc](https://huggingface.co/transformers/)).

In [ ]:
import random
seed = random.randint(0, 2**32-1)
seed

In [ ]:
examples = []

for start in SENTENCES:
    val = !python run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/$handle \
        --length 150 \
        --stop_token "{'\n'}" \
        --num_return_sequences 3 \
        --temperature 1 \
        --seed $seed \
        --prompt {'"' + start + '"'}
    generated = [val[-1-2*k] for k in range(3)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')
        examples.append([start, g])

We log the results on our previous run.

In [ ]:
# retrieve last run
project = %env WANDB_PROJECT
wandb_id = wandb.api.list_runs(project)[0]['name']

In [ ]:
# Log results on our previous wandb run
wandb.init(id=wandb_id, resume='must')
wandb.log({'examples': wandb.Table(data=examples, columns=['Input', 'Prediction'])})

# Update display name
wandb.run.name = alltweets[0].author.name
wandb.run.save()

**Results**: Open your generated "Run page" generated and look at the predictions in the "Media" panel.

You can see my trained models on my dashboard.

### [W&B Dashboard →](https://app.wandb.ai/borisd13/huggingface-twitter)

Please share your experiments and any insights you have!